In [17]:
import os

# Vérifie si le code est exécuté sur Google Colab
if 'COLAB_GPU' in os.environ:
    # Commandes à exécuter uniquement sur Google Colab
    if os.path.isdir('.git'):
        # Already in the git repository, just pull
        !git pull | grep -xq requirements.txt && pip install -r requirements.txt || echo "requirements.txt unchanged"
        # If requirements.txt changed, install dependencies
    else:
        # Clone the repository
        !git clone https://github.com/Florian-Audouard/tp-rag
        %cd tp-rag
        !pip install -r requirements.txt
else:
    # Commandes à exécuter si ce n'est pas sur Google Colab
    print("Pas sur Google Colab, ces commandes ne seront pas exécutées.")

requirements.txt unchanged


In [18]:
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import DirectoryLoader


EMBESSINGS_MODEL_NAME = "intfloat/multilingual-e5-base"
DATA_FOLDER = "data/"

In [19]:
embeddings = HuggingFaceEmbeddings(model_name=EMBESSINGS_MODEL_NAME)
vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)
loader = DirectoryLoader(DATA_FOLDER)
docs = loader.load()
print(f"Number of documents loaded: {len(docs)}")

ERROR:langchain_community.document_loaders.directory:Error loading file data/arxiv/A_Survey_of_Software-Defined_Smart_Grid_Networks_Security_Threats_and__Defense_Techniques.pdf


ImportError: unstructured package not found, please install it with `pip install unstructured`